#PROC MEANS
From [SAS 9.4 Procedures Guide - Overview: MEANS Procedure](http://support.sas.com/documentation/cdl/en/proc/67916/HTML/default/viewer.htm#n0k7qr5c2ah3stn10g1lr5oytz57.htm)

**What Does the MEANS Procedure Do?**

The MEANS procedure provides data summarization tools to compute descriptive statistics for variables across all observations and within groups of observations. For example, PROC MEANS does the following:
- calculates descriptive statistics based on moments
- estimates quantiles, which includes the median
- calculates confidence limits for the mean
- identifies extreme values
- performs a t test

By default, PROC MEANS displays output. You can also use the OUTPUT statement to store the statistics in a SAS data set. 

##Examples 1
Replicating the examples on the overview page in Python.

First example: `PROC MEANS` standard output on a dataset of integers 1 to 10.

###SAS Code

    proc means data=OnetoTen;
    run;
    
###SAS Output
<code>
                             The SAS System              

                          The MEANS Procedure

                      Analysis Variable : Integer

        N            Mean         Std Dev         Minimum         Maximum  
---------------------------------------------------------------------------
       10       5.5000000       3.0276504       1.0000000      10.0000000  
---------------------------------------------------------------------------
</code>

####Python Code

In [1]:
import pandas as pd

First, we'll create the equivalent dataset in python using the `range()` function. Then, we'll store it in a *Series*, or the 1-D data format in `pandas`.

In [2]:
OnetoTen = pd.Series(xrange(1,11))

The `.describe().` method called on a *Series* or *DataFrame* will return basic summary statistics about that object.

In [3]:
OnetoTen.describe()

count    10.00000
mean      5.50000
std       3.02765
min       1.00000
25%       3.25000
50%       5.50000
75%       7.75000
max      10.00000
dtype: float64

Perfect -- a replication that contains all the original output plus a bit more.

##Example 2
The second example on the overview is [Example 11](http://support.sas.com/documentation/cdl/en/proc/67916/HTML/default/viewer.htm#n18axszmd7up03n1densx922oe8y.htm) on the procedures example page using the `CHARITY` data set.

###SAS Code
    proc means data=Charity n mean range chartype;

        class School Year;

        var MoneyRaised HoursVolunteered;

        output out=Prize maxid(MoneyRaised(name)
            HoursVolunteered(name))= MostCash MostTime
            max= ;
          
        title 'Summary of Volunteer Work by School and Year';
    run;

    proc print data=Prize;
        title 'Best Results: Most Money Raised and Most Hours Worked';
    run;

###SAS Output
<img src="http://support.sas.com/documentation/cdl/en/proc/67916/HTML/default/images/means_ex11a.png">
<img src="http://support.sas.com/documentation/cdl/en/proc/67916/HTML/default/images/means_ex11b.png">

**Output Summary**: We have a basic descriptive summary (with range included), followed by a more detailed output that contains the names of those who raised the most money and most hours worked broken down by overall, year, school, and school/year combination

####Python Code
First we'll read in the `CHARITY` dataset and store it in the `charity` object.

In [4]:
charity = pd.read_table('../data/charity.txt',
                        delim_whitespace=True,
                        names=['School','Year','Name',
                               'MoneyRaised','HoursVolunteered'])

Let's just take a quick look to make surethe data imported correctly.

In [5]:
charity.head()

,School,Year,Name,MoneyRaised,HoursVolunteered
0,Monroe,2007,Allison,31.65,19
1,Monroe,2007,Barry,23.76,16
2,Monroe,2007,Candace,21.11,5
3,Monroe,2007,Danny,6.89,23
4,Monroe,2007,Edward,53.76,31


A good idea would be to try the `.describe()` function, but the first part of the SAS output contains something that the `.describe()` function doesn't have: the range. Instead of using that function, we'll work with `pandas` `pivot_table` objects and define our own aggregations.

At first, the syntax for `pivot_table` may seem a bit obtuse (or equally obtuse as SAS' `MAXID`), but over time I have come to favor its learned intuitiveness (Also, why is `PROC MEANS` producing anything but *means*?) The `pivot_table` function takes a *DataFrame* and aggregates according to several arguments you define.

<div class="resources">
For a comprehensive guide to <code>pivot_table</code> syntax, check out <a href="http://pbpython.com/pandas-pivot-table-explained.html">this tutorial</a>.
</div>

We have one issue before we build our `pivot_tale`: There isn't a built-in *range* function for our data. No problem -- we can write our own. With complex data manipulation, writing functions is something you'll do quite frequently, so it's important to be comfortable with the syntax.

In [6]:
def array_range(arr):
    return arr.max() - arr.min()

<div class="pynote">
<b>Python Note</b>: Python functions consist of 4 important parts: the <code>def</code> keyword indicating you're defining a function, the function name (<code>array_range</code>), any arguments to the function <code>arr</code>, and the <code>return</code> statement. 
<br><br>
<b>Detail</b>: The <code>array_range</code> function, it's going to take an array as an input, and we'll refer to that input as <code>arr</code> within the function itself. The function returns the value of the maximum minus the minimum value in that array. 
</div>

<div class="resources">
For a basic tutorial on functions, <a href="http://anh.cs.luc.edu/python/hands-on/3.1/handsonHtml/functions.html">click here</a>.
</div>

Now that we have defined the function for range, we'll create our `pivot_table`. We'll store it in an object labeled `means`. The `pivot_table` function takes 4 arguments: the *DataFrame* we're applying it to (`charity`), the levels of aggregation for the `index` (`School`, `Year`), the `values` we want to aggregate (`MoneyRaised`, `HoursVolunteered`), and how we want to aggregate them with `aggfunc` (`np.size` for *N*, `np.mean` for *mean*, `array_range` for *range*).

We'll also import the `numpy` library, as it contains several methods for aggregation we'll use.

In [7]:
import numpy as np

In [8]:
means = pd.pivot_table(charity,
                       index=['School', 'Year'],
                       values=['MoneyRaised', 'HoursVolunteered'], 
                       aggfunc=[np.size, np.mean, array_range])

In [9]:
means

size                         mean              \
             HoursVolunteered MoneyRaised HoursVolunteered MoneyRaised   
School  Year                                                             
Kennedy 2007               18          18        21.444444   29.381111   
        2008               17          17        19.411765   28.156471   
        2009               18          18        24.277778   31.579444   
Monroe  2007               16          16        18.812500   28.545000   
        2008               12          12        15.833333   28.050000   
        2009               28          28        19.142857   29.410000   

                  array_range              
             HoursVolunteered MoneyRaised  
School  Year                               
Kennedy 2007               30       39.75  
        2008               20       23.56  
        2009               15       65.44  
Monroe  2007               38       48.27  
        2008               21       52.46  
        2009               26       73.53

The next part of the output is a little more challenging. In the original SAS output, the chart has subtotals given by the `_TYPE_` column output by the `chartype` statement in `PROC MEANS`. Additionally, the [MAXID statement](http://support.sas.com/documentation/cdl/en/proc/61895/HTML/default/viewer.htm#a000146734.htm) is used to output a dataset called `Prize` which contains the names of the students who match the maximum values for `MoneyRaised` and `HoursVolunteered`.  

We'll break this problem up into two parts:
1. We need to roll up our pivot_table to the year, school, and total level -- essentially, add subtotals. 
2. We need to find the person's name who sold the most and volunteered within the different levels.

Let's start with #1.  

**Subtotals**  

Adding a `margins=True` argument to `pivot_table` will add in totals for our aggregation functions.

In [10]:
Prize = pd.pivot_table(charity,
                       index=['School', 'Year'],
                       values=['MoneyRaised', 'HoursVolunteered'],
                       aggfunc=[np.size, np.mean, array_range], 
                       margins=True)

In [11]:
Prize

size                         mean              \
             HoursVolunteered MoneyRaised HoursVolunteered MoneyRaised   
School  Year                                                             
Kennedy 2007               18          18        21.444444   29.381111   
        2008               17          17        19.411765   28.156471   
        2009               18          18        24.277778   31.579444   
Monroe  2007               16          16        18.812500   28.545000   
        2008               12          12        15.833333   28.050000   
        2009               28          28        19.142857   29.410000   
All                       109         109        20.000000   29.291284   

                  array_range              
             HoursVolunteered MoneyRaised  
School  Year                               
Kennedy 2007               30       39.75  
        2008               20       23.56  
        2009               15       65.44  
Monroe  2007               38       48.27  
        2008               21       52.46  
        2009               26       73.53  
All                        38       73.53

However, this isn't quite what we want -- we also want subtotals by overall `Year` and `School` levels. 

There is a bit of a workaround for this: what we're going to do is switch one of our `index` argument values to a `column` argument value. 

You might ask "How did you know to do this?" to which I would respond "Google." Someone has already asked this [exact question](http://stackoverflow.com/questions/15570099/pandas-pivot-tables-row-subtotals) on StackOverflow, and the author of the `pandas` package himself answered. I doubt you'll get someone who worked on `PROC MEANS` answering your questions about it.

In [12]:
Prize = pd.pivot_table(charity,
                       index=['School'],
                       columns=['Year'], 
                       values=['MoneyRaised', 'HoursVolunteered'],
                       aggfunc=[np.size, np.mean, array_range], 
                       margins=True)

In [13]:
Prize

size                                                 \
        MoneyRaised                HoursVolunteered                  
Year           2007 2008 2009  All             2007 2008 2009  All   
School                                                               
Kennedy          18   17   18   53               18   17   18   53   
Monroe           16   12   28   56               16   12   28   56   
All              34   29   46  109               34   29   46  109   

               mean            ...                             array_range  \
        MoneyRaised            ... HoursVolunteered            MoneyRaised   
Year           2007       2008 ...             2009        All        2007   
School                         ...                                           
Kennedy   29.381111  28.156471 ...        24.277778  21.754717       39.75   
Monroe    28.545000  28.050000 ...        19.142857  18.339286       48.27   
All       28.987647  28.112414 ...        21.152174  20.000000       48.27   

                                                             
                             HoursVolunteered                
Year      2008   2009    All             2007 2008 2009 All  
School                                                       
Kennedy  23.56  65.44  65.44               30   20   15  30  
Monroe   52.46  73.53  73.53               38   21   26  38  
All      52.46  73.53  73.53               38   26   26  38  

[3 rows x 24 columns]

Almost there -- we can use the `.stack()` [function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.stack.html) to pivot a column label to a row label.

*(This is also a part of the StackOverflow answer)*

In [14]:
Prize.stack('Year')

size                         mean              \
             HoursVolunteered MoneyRaised HoursVolunteered MoneyRaised   
School  Year                                                             
Kennedy 2007               18          18        21.444444   29.381111   
        2008               17          17        19.411765   28.156471   
        2009               18          18        24.277778   31.579444   
        All                53          53        21.754717   29.734906   
Monroe  2007               16          16        18.812500   28.545000   
        2008               12          12        15.833333   28.050000   
        2009               28          28        19.142857   29.410000   
        All                56          56        18.339286   28.871429   
All     2007               34          34        20.205882   28.987647   
        2008               29          29        17.931034   28.112414   
        2009               46          46        21.152174   30.258913   
        All               109         109        20.000000   29.291284   

                  array_range              
             HoursVolunteered MoneyRaised  
School  Year                               
Kennedy 2007               30       39.75  
        2008               20       23.56  
        2009               15       65.44  
        All                30       65.44  
Monroe  2007               38       48.27  
        2008               21       52.46  
        2009               26       73.53  
        All                38       73.53  
All     2007               38       48.27  
        2008               26       52.46  
        2009               26       73.53  
        All                38       73.53

**Matching Names to maximum `MoneyRaised` and `HoursVolunteered` values**  
*i.e. the `MAXID` statement in SAS*

The other part of the output contains the names that match the maximum values of our two numeric columns for each level of aggregation. We'll again define an aggregation function to do this for us.

In [15]:
def name_max(arr):
    return charity['Name'].ix[arr.idxmax()]

The function above applies a few new concepts. First, inside the `[]` brackets, we're going to find the index of the maximum value of an array `arr`. The `.ix` is an indexing operator that will return a value based on an index. We are then going to find the `Name` that matches to that index value in the `charity['Name']` series. 

<div class="resources">
For more on indexing and selecting data with `pandas`: <a href="http://pandas.pydata.org/pandas-docs/stable/indexing.html">click here</a>.
</div>

Finally, we'll roll up all the concepts we've applied to one single output. To review:

- We used `pivot_table` to create tables based on defined levels of aggregation and specific aggregation functions
- We created our own *range* function with `array_range()`
- We added subtotals with `margins=True` in a `pivot_table`
- We found a suitable workaround for subtotals on different levels of detail using `columns=` followed by a `.stack()` function
- We created our own `MAXID` equivalent with `name_max()`

In [16]:
Prize = pd.pivot_table(charity,
                       index=['School'],
                       columns=['Year'],
                       values=['MoneyRaised', 'HoursVolunteered'],
                       aggfunc=[np.size, np.mean, array_range, name_max],
                       margins=True)

In [17]:
Prize.stack('Year')

size                         mean              \
             HoursVolunteered MoneyRaised HoursVolunteered MoneyRaised   
School  Year                                                             
Kennedy 2007               18          18        21.444444   29.381111   
        2008               17          17        19.411765   28.156471   
        2009               18          18        24.277778   31.579444   
        All                53          53        21.754717   29.734906   
Monroe  2007               16          16        18.812500   28.545000   
        2008               12          12        15.833333   28.050000   
        2009               28          28        19.142857   29.410000   
        All                56          56        18.339286   28.871429   
All     2007               34          34        20.205882   28.987647   
        2008               29          29        17.931034   28.112414   
        2009               46          46        21.152174   30.258913   
        All               109         109        20.000000   29.291284   

                  array_range                     name_max              
             HoursVolunteered MoneyRaised HoursVolunteered MoneyRaised  
School  Year                                                            
Kennedy 2007               30       39.75              Jay      Thelma  
        2008               20       23.56              Amy        Bill  
        2009               15       65.44          Che-Min      Luther  
        All                30       65.44              Jay      Luther  
Monroe  2007               38       48.27            Tonya       Tonya  
        2008               21       52.46           Myrtle     Cameron  
        2009               26       73.53             L.T.     Willard  
        All                38       73.53            Tonya     Willard  
All     2007               38       48.27            Tonya       Tonya  
        2008               26       52.46              Amy     Cameron  
        2009               26       73.53             L.T.     Willard  
        All                38       73.53            Tonya     Willard

You may have to scroll to the right in the output window to see all the data. If we just want to print the non-formatted table, we can prepend `print` to our statement. (Or use the `print()` function in Python 3)

In [18]:
print Prize.stack('Year')

                         size                         mean              \
             HoursVolunteered MoneyRaised HoursVolunteered MoneyRaised   
School  Year                                                             
Kennedy 2007               18          18        21.444444   29.381111   
        2008               17          17        19.411765   28.156471   
        2009               18          18        24.277778   31.579444   
        All                53          53        21.754717   29.734906   
Monroe  2007               16          16        18.812500   28.545000   
        2008               12          12        15.833333   28.050000   
        2009               28          28        19.142857   29.410000   
        All                56          56        18.339286   28.871429   
All     2007               34          34        20.205882   28.987647   
        2008               29          29        17.931034   28.112414   
        2009               46         

---

In [19]:
# This cell imports the styling for this notebook. You can safely ignore it.

from IPython.display import HTML

def css_styling():
    styles = open("../_styles/custom.css", "r").read()
    return HTML(styles)
css_styling()